In [1]:
!pip install owlready2 orkg habanero thefuzz semanticscholar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 47.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 23.8 MB/s eta 0:00:00
  Created wheel for owlready2: filename=Owlready2-0.41-cp310-cp310-linux_x86_64.whl size=24170148 sha256=35e55341db38ac98b0c742b9f1f1565e9a94faa06bc7fa7dcb69886042ae3189
  Stored in directory: /root/.cache/pip/wheels/da/02/c0/a65e08daab6da8123998823c9aee7bba656b1500ab05d183ef
  Created wheel for cardinality: filename=cardinality-0.1.1-py3-none-any.whl size=2586 sha256=7f2f2170cb6f6c81b80f51b2790f7acd5be638fa56e6a9e9c7a994381381b43e
  Stored in directory: /root/.cache/pip/wheels/b8/19/d1/2665c004b583a7d1880fa59055a3e462d6e35841a01b57010b
  Created wheel for hammock: filename=hammock-0.2.4-py3-none-any.whl size

In [3]:
import pandas as pd
from orkg import ORKG # import base class from package
orkg = ORKG(host="https://incubating.orkg.org", creds=('nlp4re@example.com', '123456'))
from habanero import Crossref
cr = Crossref()
import numpy as np
from thefuzz import fuzz
from thefuzz import process
from semanticscholar import SemanticScholar
sch = SemanticScholar()
import pickle as cPickle
import math
import json
import glob
def readJson(path):
    # Opening JSON file
    f = open(path)
    # returns JSON object as 
    # a dictionary
    return json.load(f)
def createOrFindClass(label):
    findClass = orkg.classes.get_all(q=label, exact=True).content
    if (len(findClass) > 0):
        classID = findClass[0]['id']
    else:
        classID = orkg.classes.add(label=label).content['id']
    return classID
def createOrFindPredicate(label):
    findPredicate = orkg.predicates.get(q=label, exact=True).content
    if (len(findPredicate) > 0):
        predicate = findPredicate[0]['id']
    else:
        predicate = orkg.predicates.add(label=label).content['id']
    return predicate
def shouldCreateStatement(subjectid, predicateid):
    statements = orkg.statements.get_by_subject_and_predicate(subject_id=subjectid, predicate_id=predicateid, size=5).content
    if (len(statements) > 0):
        return False
    else:
        return True
def shouldCreateTemplateForClass(idc):
    statements = orkg.statements.get_by_object_and_predicate(object_id=idc, predicate_id='sh:targetClass', size=5).content
    if (len(statements) > 0):
        return False
    else:
        return True

/usr/local/lib/python3.10/dist-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
# To load the objects 
paperObjects = cPickle.load(open('/content/drive/MyDrive/Colab Notebooks/NLP4RE/papersMetadata.p', 'rb'))

In [6]:
mappingName = []

In [7]:
def defineOntology(cID, cLabel, parsedJson):
    for attribute in parsedJson['attributes']:
        pid = orkg.predicates.add(label=attribute['name'] if attribute['name']!='id' else 'external id').content['id']
        mappingName.append([cID,'predicate',attribute['name'],pid, None])
        if ('description' in attribute):
            lip = orkg.literals.add(label=attribute['description'], datatype='xsd:string').content['id']
            orkg.statements.add(subject_id=pid, predicate_id='description', object_id=lip)
        # add property shape
        propertyShapeID = orkg.resources.add(label='Property shape for '+templateID, classes=['PropertyShape']).content['id']
        orkg.statements.add(subject_id=templateID, predicate_id='sh:property', object_id=propertyShapeID)
        orkg.statements.add(subject_id=propertyShapeID, predicate_id='sh:path', object_id=pid)
        # Maximum count
        limaxc = orkg.literals.add(label=1, datatype='xsd:integer').content['id']
        orkg.statements.add(subject_id=propertyShapeID, predicate_id='sh:maxCount', object_id=limaxc)
        # if it's greater than 2 it means not boolean and a class has to be created
        if ('dimensions' not in attribute) and ('characteristics' in attribute) and len(attribute['characteristics']) > 2: 
            cclassLabel = cLabel+' '+attribute['name']
            cclassID = createOrFindClass(cclassLabel)
            orkg.statements.add(subject_id=propertyShapeID, predicate_id='sh:class', object_id=cclassID)
            mappingName[-1:][0][-1] = cclassID
            print(cclassID)
            mappingName.append([classID,'class',cclassLabel,cclassID, None])
            for characteristic in attribute['characteristics']:
                rid = orkg.resources.add(label=characteristic['value'], classes=[cclassID]).content['id']
                mappingName.append([cclassID,'resource',characteristic['value'],rid, None])
                if ('description' in characteristic):
                    lir = orkg.literals.add(label=characteristic['description'], datatype='xsd:string').content['id']
                    orkg.statements.add(subject_id=rid, predicate_id='description', object_id=lir)
        # it's a booolean range
        elif ('dimensions' not in attribute) and (('characteristics' in attribute) and len(attribute['characteristics']) == 2):
            orkg.statements.add(subject_id=propertyShapeID, predicate_id='sh:datatype', object_id='Boolean')
        elif ('dimensions' in attribute):
            cclassLabel = cLabel+' '+attribute['name']
            cclassID = createOrFindClass(cclassLabel)
            mappingName[-1:][0][-1] = cclassID
            mappingName.append([cID,'class',cclassLabel,cclassID, None])
            # Create template
            dtemplateID = orkg.resources.add(label=cclassLabel, classes=['NodeShape']).content['id']
            orkg.statements.add(subject_id=dtemplateID, predicate_id='sh:targetClass', object_id=cclassID)
            if ('characteristics' in attribute) and len(attribute['characteristics']) > 2:
                rclassLabel = cclassLabel+' range'
                rclassID = createOrFindClass(rclassLabel)
                for characteristic in attribute['characteristics']:
                    rid = orkg.resources.add(label=characteristic['value'], classes=[rclassID]).content['id']
                    mappingName.append([rclassID,'resource',characteristic['value'],rid, None])
                    if ('description' in characteristic):
                        lir = orkg.literals.add(label=characteristic['description'], datatype='xsd:string').content['id']
                        orkg.statements.add(subject_id=rid, predicate_id='description', object_id=lir)
            for dimension in attribute['dimensions']:
                pid = orkg.predicates.add(label=dimension['value']).content['id']
                mappingName.append([cclassID,'predicate',dimension['value'],pid, None])
                if ('description' in dimension):
                    lip = orkg.literals.add(label=dimension['description'], datatype='xsd:string').content['id']
                    orkg.statements.add(subject_id=pid, predicate_id='description', object_id=lip)
                 # add property shape
                dpropertyShapeID = orkg.resources.add(label='Property shape for '+dtemplateID, classes=['PropertyShape']).content['id']
                orkg.statements.add(subject_id=dtemplateID, predicate_id='sh:property', object_id=dpropertyShapeID)
                orkg.statements.add(subject_id=dpropertyShapeID, predicate_id='sh:path', object_id=pid)
                if ('characteristics' in attribute) and len(attribute['characteristics']) > 2:
                    mappingName[-1:][0][-1] = rclassID
                    orkg.statements.add(subject_id=dpropertyShapeID, predicate_id='sh:class', object_id=rclassID)
                    limaxc = orkg.literals.add(label=1, datatype='xsd:integer').content['id']
                    orkg.statements.add(subject_id=dpropertyShapeID, predicate_id='sh:maxCount', object_id=limaxc)
                elif len(attribute['characteristics']) == 2:
                    mappingName[-1:][0][-1] = 'Boolean'
                    orkg.statements.add(subject_id=dpropertyShapeID, predicate_id='sh:datatype', object_id='Boolean')
                    limaxc = orkg.literals.add(label=1, datatype='xsd:integer').content['id']
                    orkg.statements.add(subject_id=dpropertyShapeID, predicate_id='sh:maxCount', object_id=limaxc)
            orkg.statements.add(subject_id=propertyShapeID, predicate_id='sh:class', object_id=cclassID)
        # it's a string range
        elif (('unique' in attribute) and attribute['unique'] == True):
            mappingName[-1:][0][-1] = 'String'
            orkg.statements.add(subject_id=propertyShapeID, predicate_id='sh:datatype', object_id='String')
        if('mandatory' in attribute and attribute['mandatory'] == True):
            liminc = orkg.literals.add(label=1, datatype='xsd:integer').content['id']
            orkg.statements.add(subject_id=propertyShapeID, predicate_id='sh:minCount', object_id=liminc)

# Descriptions

In [8]:
jsonData = readJson('/content/drive/MyDrive/Colab Notebooks/NLP4RE/data/rqfo-data/structure/o1/t1/description.json')
print(jsonData['name'])
classLabel = 'NLP4RE '+jsonData['name']
classID = createOrFindClass(classLabel)
if shouldCreateStatement(classID,'description'):
    lic = orkg.literals.add(label=jsonData['description'], datatype='xsd:string').content['id']
    orkg.statements.add(subject_id=classID, predicate_id='description', object_id=lic)
print(classID)
## Create Template for the class
if shouldCreateTemplateForClass(classID):
    templateID = orkg.resources.add(label=classLabel, classes=['NodeShape']).content['id']
    orkg.statements.add(subject_id=templateID, predicate_id='sh:targetClass', object_id=classID)
print(templateID)
defineOntology(classID,classLabel,jsonData)

Description
C44000
R510000


# Factors

In [9]:
jsonData = readJson('/content/drive/MyDrive/Colab Notebooks/NLP4RE/data/rqfo-data/structure/o1/t1/factor.json')
print(jsonData['name'])
classLabel = 'NLP4RE '+jsonData['name']
classID = createOrFindClass(classLabel)
if shouldCreateStatement(classID,'description'):
    lic = orkg.literals.add(label=jsonData['description'], datatype='xsd:string').content['id']
    orkg.statements.add(subject_id=classID, predicate_id='description', object_id=lic)
print(classID)
## Create Template for the class
if shouldCreateTemplateForClass(classID):
    templateID = orkg.resources.add(label=classLabel, classes=['NodeShape']).content['id']
    orkg.statements.add(subject_id=templateID, predicate_id='sh:targetClass', object_id=classID)
print(templateID)
defineOntology(classID,classLabel,jsonData)

Quality Factor
C44001
R510007
C44004
C44005


# Approachs

In [10]:
jsonData = readJson('/content/drive/MyDrive/Colab Notebooks/NLP4RE/data/rqfo-data/structure/o1/t1/approach.json')
print(jsonData['name'])
classLabel = 'NLP4RE '+jsonData['name']
classID = createOrFindClass(classLabel)
if shouldCreateStatement(classID,'description'):
    lic = orkg.literals.add(label=jsonData['description'], datatype='xsd:string').content['id']
    orkg.statements.add(subject_id=classID, predicate_id='description', object_id=lic)
print(classID)
## Create Template for the class
if shouldCreateTemplateForClass(classID):
    templateID = orkg.resources.add(label=classLabel, classes=['NodeShape']).content['id']
    orkg.statements.add(subject_id=templateID, predicate_id='sh:targetClass', object_id=classID)
print(templateID)
defineOntology(classID,classLabel,jsonData)

Approach
C44006
R510050
C44007
C44008


# Datasets

In [11]:
jsonData = readJson('/content/drive/MyDrive/Colab Notebooks/NLP4RE/data/rqfo-data/structure/o1/t1/dataset.json')
print(jsonData['name'])
classLabel = 'NLP4RE '+jsonData['name']
classID = createOrFindClass(classLabel)
if shouldCreateStatement(classID,'description'):
    lic = orkg.literals.add(label=jsonData['description'], datatype='xsd:string').content['id']
    orkg.statements.add(subject_id=classID, predicate_id='description', object_id=lic)
print(classID)
## Create Template for the class
if shouldCreateTemplateForClass(classID):
    templateID = orkg.resources.add(label=classLabel, classes=['NodeShape']).content['id']
    orkg.statements.add(subject_id=templateID, predicate_id='sh:targetClass', object_id=classID)
print(templateID)
defineOntology(classID,classLabel,jsonData)

Data Set
C44012
R510098
C44013
C44014
C44015
C44016


In [12]:
mappingDF =pd.DataFrame(mappingName, columns=['CLASS_ID','TYPE', 'LABEL','ORKG_ID','Range'])
mappingDF.to_csv('/content/drive/MyDrive/Colab Notebooks/NLP4RE/mappingDF.csv',index=False)

In [13]:
mappingDF

,CLASS_ID,TYPE,LABEL,ORKG_ID,Range
0,C44000,predicate,id,P84000,String
1,C44000,predicate,qf id,P84001,None
2,C44000,predicate,definition,P84002,None
3,C44000,predicate,impact,P84003,None
4,C44000,predicate,empirical evidence,P84004,None
...,...,...,...,...,...
136,C44016,resource,no link,R510132,None
137,C44016,resource,upon request,R510133,None
138,C44016,resource,private,R510134,None
139,C44016,resource,proprietary,R510135,None


In [14]:
mappingName

[['C44000', 'predicate', 'id', 'P84000', 'String'],
 ['C44000', 'predicate', 'qf id', 'P84001', None],
 ['C44000', 'predicate', 'definition', 'P84002', None],
 ['C44000', 'predicate', 'impact', 'P84003', None],
 ['C44000', 'predicate', 'empirical evidence', 'P84004', None],
 ['C44000', 'predicate', 'practitioners involved', 'P84005', None],
 ['C44001', 'predicate', 'id', 'P84006', 'String'],
 ['C44001', 'predicate', 'name', 'P84007', 'String'],
 ['C44001', 'predicate', 'aspect', 'P84008', 'C44002'],
 ['C44001', 'class', 'NLP4RE Quality Factor aspect', 'C44002', None],
 ['C44003', 'resource', '-', 'R510012', None],
 ['C44003', 'resource', '+', 'R510013', None],
 ['C44003', 'resource', '?', 'R510014', None],
 ['C44003', 'resource', '', 'R510015', None],
 ['C44002', 'predicate', 'adequacy', 'P84009', 'C44003'],
 ['C44002', 'predicate', 'atomicity', 'P84010', 'C44003'],
 ['C44002', 'predicate', 'completeness', 'P84011', 'C44003'],
 ['C44002', 'predicate', 'compliance', 'P84012', 'C44003'],